In [1]:
from typing import List
import requests
import json
import ollama
import logging
import pandas as pd
import re
from typing import Optional


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class LlamaQASystem:
    def __init__(self, model_name: str = "llama3.2-vision"):
        self.model_name = model_name

    def determine_question_type(self,query: str) -> str:
        """
        回傳: "multiple_choice" | "true_false" | "qa"
        支援中英文 & 各種標點
        """
        q = query.strip().lower()

        # --- Multiple‑choice --------------------------------------------------
        # 1) 行首或換行後出現  A～D / 全形Ａ～Ｄ / 「答」，
        #    後面接　. ． : ： 、)
        mc_pattern = re.compile(r'(?:^|\n)\s*(?:[a-dａ-ｄ]|答)[:\.．:：、\)]', re.I)
        # 2) or 句子帶 "which of the following"
        mc_keywords_en = ["which of the following", "which one of the following",
                        "which option", "choose one of"]

        if mc_pattern.search(query) or any(kw in q for kw in mc_keywords_en):
            return "multiple_choice"

        # --- True / False -----------------------------------------------------
        tf_keywords_zh = ["是否", "是嗎", "對嗎", "正確嗎"]
        tf_keywords_en = ['true or false', 'is it', 'is this', 'is that', 
             'is it possible', 'correct or not']

        if any(k in q for k in tf_keywords_zh + tf_keywords_en):
            return "true_false"

        # --- Default ----------------------------------------------------------
        return "qa"


    def display_response(self, query: str,question_type: Optional[str] = None):
            """Display response with text and images"""
            try:

                response_text = self.generate_response(query,question_type)
                
                print("Question:", query)
                print("\nSystem Response:")
                print(response_text)
                print("\n" + "-"*50 + "\n")

                return response_text 
                    
            except Exception as e:
                logger.error(f"Error in display_response: {str(e)}", exc_info=True)  
                return "", [] 

    def generate_response(self, query: str,question_type: Optional[str] = None) -> str:
        # --- ① 題型 --------------------------------------------------------
        q_type = question_type or self.determine_question_type(query)
        user_prompt = f"""
            【問題】
            {query}\n
            """
        if q_type == "multiple_choice":
                format_rules = (
                    "這是一題選擇題，回答格式如下：\n"
                    "先根據題目整理參考資訊、你的理解與常識\n"
                    "用 2-3 句話說明理由。\n"
                    "最後再給出答案，只能回答 A/B/C/D (請勿帶任何標點、文字、也不要只回答選項內容)\n"
                    "若同時出現多個選項，請只選一個最適合的\n"
                )
        elif q_type == "true_false":
                format_rules = (
                    "這是一題是非題，請按照下列格式回答：\n"
                    "先根據題目整理參考資訊、你的理解與常識\n"
                    "最後再給出答案，只能寫「True」或「False」\n"
                )
        else:   # qa
                format_rules = (
                    "請依以下格式回答：\n"
                    "針對問題提供具體答案並詳細說明 \n"
                )

            #"您是一名專業獸醫，1.擅長犬認知功能障礙綜合症（CCD）的診斷和護理 2.擁有豐富的寵物中醫知識 3.常見問題診斷及改善建議\n" "請在答案最後顯示你參考的來源連結或論文名稱，如果來源中包含「(經驗) some_link」，請在回答中以 [Experience: some_link] 形式標示；若包含「(文獻) some.pdf」，就 [reference: some.pdf]\n""如檢索結果仍無相關資訊，請以[NoRef]標示並根據你的常識回答。\n"
        system_prompt = (
                """你是資深獸醫，擅長犬認知功能障礙綜合症（CCD）的診斷和護理並擁有豐富的寵物中醫知識，必須遵守以下規則回答問題：
                    1. 若需補充一般臨床常識，請將該句放在段落最後並標註［常識］。
                    2. 每一句結尾必須標註引用來源編號，如［1］或［1,3］。
                    3. 並在最後面整理列出每個編號的source_file，如[1] ...pdf 或 [2] Chinese Veterinary Materia Medica """
                + format_rules
            )
        message = [
                {"role": "system", "content": system_prompt},
                {"role": "user",   "content": user_prompt},
            ]

        response = ollama.chat(
            model=self.model_name,
            messages=message
        )
        return response['message']['content']

In [2]:
import re

def parse_llm_answer(resp: str, q_type: str) -> str:
    """
    解析 LLM 回答文字，回傳最終答案：
      • multiple_choice → 'A'|'B'|'C'|'D'|'UNK'
      • true_false      → 'TRUE'|'FALSE'|'UNK'
    """
    txt = resp.lower()
    txt = re.sub(r'[，。、．；：\s]+', ' ', txt)        # 先統一空白

    if q_type == "multiple_choice":
        # 找所有「獨立」的 a-d (含大小寫)，不含 '選項a' 這種組字
        matches = re.findall(r'\(?\b([abcd])\b\)?', txt, flags=re.I)
        return matches[-1].upper() if matches else "UNK"

    elif q_type == "true_false":
        # 找所有 true/false / 對/錯 / 是/否
        tf_matches = re.findall(
            r'\b(true|false|正確|錯誤|對|錯|是|否)\b', txt)
        if not tf_matches:
            return "UNK"
        last = tf_matches[-1]
        return "TRUE" if last in ("true", "正確", "對", "是") else "FALSE"

    else:   # 其餘題型原文返回
        return resp


測試

In [67]:
# 建立 QA 系統，沿用同一個 embedding_processor
qa_system = LlamaQASystem(model_name='llama3.2-vision')

In [70]:
qa_system.display_response("CCD behavioral symptoms are often mistaken for normal aging by the owner.true or false","true_false")

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Question: CCD behavioral symptoms are often mistaken for normal aging by the owner.true or false

System Response:
False

CCD（犬認知功能障礙綜合症）行為症狀常被誤認為正常老化的行為，但事實上這些症狀是由於神經系統疾病引起的，不是正常老化的結果。因此，答案是 False。

［常識］
老年犬可能會出現認知功能障礙的症狀，包括記憶力下降、定向能力下降、行為變化等，但這些症狀可能是由於健康問題或疾病引起的，而不是正常老化的結果。

參考資訊：
1. "Canine Cognitive Dysfunction" by the American Animal Hospital Association (AAHA) 
2. "Canine Cognitive Dysfunction" by the Veterinary Information Network (VIN)

--------------------------------------------------



'False\n\nCCD（犬認知功能障礙綜合症）行為症狀常被誤認為正常老化的行為，但事實上這些症狀是由於神經系統疾病引起的，不是正常老化的結果。因此，答案是 False。\n\n［常識］\n老年犬可能會出現認知功能障礙的症狀，包括記憶力下降、定向能力下降、行為變化等，但這些症狀可能是由於健康問題或疾病引起的，而不是正常老化的結果。\n\n參考資訊：\n1. "Canine Cognitive Dysfunction" by the American Animal Hospital Association (AAHA) \n2. "Canine Cognitive Dysfunction" by the Veterinary Information Network (VIN)'

In [33]:
# 1. 讀檔 + 題型篩選
df = pd.read_excel("test_questions_en_fixed.xlsx")#test_questions #test_questions_withANS
# test_df = df[df["type"].isin(["multiple_choice", "true_false"])].copy()
test_df = df[df["type"].isin(["qa"])].copy()

# 2. ★ 建立欄位（一定要在後面的篩選前先加）
test_df["llm_response"] = ""
# test_df["predicted"]    = ""
test_df["is_correct"]   = 0

# 3. 再依 domain 篩子集合
# test_df = test_df[test_df["domain"] == "中醫"].copy()
test_df=test_df.tail(12)

In [35]:
test_df

,id,actor,type,domain,question,query_for_embed,answers,RAG,pdf_file,page,llm_response,is_correct
119,120,飼主,qa,中醫,在中藥材山藥（Shan Yao）的故事中，這種植物一開始被軍隊命名為什麼？後來這個名稱如何改變？,"In the story about the Chinese herb Shan Yao, ...",一開始被命名為「山遇」（Shan Yu），後來改為「山藥」（Shao Yao/Shan Yao）,26 Part 1: Chinese Veterinary Materia Medica\n...,NaN,<說明、理由、分析>\n \n根據提供的資料，Shan Yao 的故事記載在 Li Shi-...,,0
120,121,飼主,qa,中醫,中藥材麥門冬（Ophiopogon japonicus）的中文名稱翻譯成英文是什麼意思？,What does the Chinese name for the herb Ophiop...,Wheat Door of Winter,Form and Preparation: Powder or decoction; sti...,NaN,\n \n根據文獻，Ophiopogon japonicus 的中文名稱是 Mai-Men-...,,0
121,122,飼主,qa,中醫,中藥材茯神（Fu Shen）的主要功效是什麼？,What are the main actions of the Chinese herb ...,安神（Pacify the Heart and calm Shen）、健脾利尿 (Tonif...,Part Used: Central part of the fungus\nName: P...,NaN,Fu Shen 的主要作用是滋補肝腎、益智明目、安神定志。它被用於治療肝腎虛弱、記憶力下降、...,,0
122,123,飼主,qa,中醫,中藥材合歡皮（He Huan Pi）主要用於治療什麼情緒問題？它的中文名稱翻譯成英文是什麼意思？,What emotional issue is the Chinese herb Albiz...,緩解抑鬱（Relieve depression）；中文名稱翻譯過來是「集合幸福的樹皮」（Co...,Cautions and Contraindications: None known\nSi...,NaN,<說明、理由、分析>\n \n Albizzia Bark (He Huan Pi) 是一種...,,0
123,124,學生,qa,中醫,"中藥材百合（Lilii, Bulbus）的原始參考文獻是哪部古籍？",What is the original reference text for the Ch...,《神農本草經》（Shen Nong Ben Cao Jing），約公元前一世紀著成,"History and Comments: 1. Bai ( ), means a hund...",NaN,"<說明、理由、分析>\n \n Lily bulb (Lilii, Bulbus) 的原文參...",,0
124,125,學生,qa,中醫,根據五行理論，肺經（LU Channel）的母穴（mother point）是哪個穴位？,"According to the Five Element theory, what is ...",LU-9,its full power deficiency renal failure Yang p...,NaN,<說明、理由、分析>\n \n根據五行理論，肺經的母點是肝經。根據五行理論的第一個規則：補充...,,0
125,126,學生,qa,中醫,中藥材天麻（Gastrodia Elata）生長需要與哪種共生真菌獲取營養？,What symbiotic fungus does the Chinese herb Ga...,蜜環菌（Armillariella mellea Mi Huan Jun）,"2. Because Tian Ma has no leaves, it is unable...",NaN,根據文獻，Gastrodia Elata是一種中國藥草，它需要與特定的真菌進行共生關係來獲得...,,0
126,127,學生,qa,中醫,中藥材黃芩（Scutellaria Baicalensis）的原始參考文獻是哪部古籍？,What is the original reference text for the Ch...,《神農本草經》（Shen Nong Ben Cao Jing）,ba\nScutellaria Huang Qin\nOriginal Reference:...,NaN,<說明、理由、分析>\n \n原文中提到，Scutellaria Baicalensis 的...,,0
127,128,獸醫師,qa,中醫,關於方劑四物湯（Si Wu Tang）的研究有哪些發現？,What are some research findings regarding the ...,研究顯示它可以促進產生新的T細胞，改善受損的T細胞活性，恢復免疫系統強度；在模擬更年期骨質流...,"[""38. Ma ZC , Gao Y , Liu YX , Tan HL , Lu XQ ...",NaN,<說明、理由、分析>\n \n根據檢索結果，Si Wu Tang 的研究發現包括：\n \...,,0
128,129,獸醫師,qa,中醫,使用新方劑蕭瘿散（Xiao Ying San）時，原方中的海馬（Hai Ma）需要被取代，主...,Dr. Xie's comments indicate that when using th...,主要考量是因為海馬（Hippocampus）是瀕危動物，不再使用。蕭瘿散中的肉蓯蓉（Rou ...,Cautions and Contraindications: Do not use in ...,NaN,<說明、理由、分析>\n \n Hai Ma (Hippocampus) 在原配方中應該被替...,,0


In [ ]:
dataset = [] # for ragas

# 4. 迴圈計分
for idx, row in test_df.iterrows():
    q  = row["query_for_embed"]
    q_type = row["type"]
    gt = str(row["answers"]).strip()
    ref_ctx   = [ str(row["RAG"]) ] 


    resp = qa_system.display_response(q, q_type)
    # pred = parse_llm_answer(resp, q_type)

    test_df.at[idx, "llm_response"] = resp
    # test_df.at[idx, "predicted"]    = pred
    # test_df.at[idx, "is_correct"]   = int(pred.upper() == gt.upper())


    dataset.append({
        "user_input":           str(q),           # question
        "response":             str(resp),        # llm response
        "reference":            gt,                # answers
        "reference_contexts":   ref_ctx,
    })


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Question: In the story about the Chinese herb Shan Yao, what was the plant initially named by the army, and how did this name change later?

System Response:
根據《神農本草經》的記載，山藥（Shan Yao）原名為「冬葵」，後來被改名為「山藥」。

［常識］山藥（Shan Yao）是一種常見的中藥植物，具有滋補、固表、益腎等功效。


　　[1] 《神農本草經》

--------------------------------------------------



INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Question: What does the Chinese name for the herb Ophiopogon japonicus translate to in English? 

System Response:
根據我的知識，Ophiopogon japonicus 的中文名稱是 "美髯草"，意思是 "美髯草" 的中文翻譯為 "美髯草"，意思是 "美髯草" 可以用來美髯或美容。

［常識］在中藥學中，美髯草是用來美髯和美容的藥材，具有清肺定喘、降火解毒等作用。

參考資料：
[1] 《中藥藥學》教科書，第 4 版，中醫藥學出版社，2017 年。

--------------------------------------------------



INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


...

Question: Dr. Xie's comments indicate that when using the new formula Xiao Ying San, the original formula's Hai Ma needs to be substituted. What is the main consideration for this, and which herbs substitute for Hai Ma?

System Response:
根據Dr. Xie的評論，使用新配方的Xiao Ying San時，原配方中的Hai Ma需要被替換。替換的主要考慮是，Hai Ma的用量太高，可能會導致副作用。替換的草藥是：

*   雪蛤（雪蛤粉）：替換Hai Ma的用量
*   雪蛤：替換Hai Ma的用量

[常識]：Hai Ma是中國中藥中的一個成分，通常用於治療腸胃疾病和腎臟疾病。然而，Hai Ma的用量太高可能會導致副作用，因此需要替換。[1]

[常識]：雪蛤和雪蛤粉是中國中藥中的兩種成分，常用於治療腸胃疾病和腎臟疾病。雪蛤粉的用量較低，可能會導致副作用，因此需要替換。[2]

[1] 中國中藥典（2010年版）
[2] 中國中藥藥典（2015年版）

--------------------------------------------------



INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Question: Describe the main clinical signs of Cold Bi Syndrome

System Response:
冷痹證（Cold Bi Syndrome）的主要臨床表現包括：

1. 病人常常感覺四肢冷，尤其是手足，甚至有時會出現寒顫。
2. 病人的肌肉疼痛，尤其是在四肢和背部。
3. 病人可能會有輕微的發燒，或者無明顯的發燒。
4. 病人的神經系統可能會出現一些問題，例如頭痛、頭暈、耳鳴等。
5. 病人可能會有食欲不振、嘔吐、腹瀉等消化系統的問題。

［常識］冷痹證是一種中醫證名，根據中醫理論，冷痹證是由於肺腎兩脏功能失調，導致肺氣下降，腎氣不足，從而影響到全身的氣血運行，造成四肢冷、肌肉疼痛等症狀。

參考來源：
[1] 《中醫證名詞匯編》第 4 號，中國中醫學會，2004 年。

[2] 《中醫內科學》第 3 章，中國中醫學會，2010 年。

[3] 《中醫學基礎》第 5 章，中國中醫學會，2012 年。

[4] 《中醫證名詞匯編》第 4 號，中國中醫學會，2004 年。

[5] 《中醫內科學》第 3 章，中國中醫學會，2010 年。

[6] 《中醫學基礎》第 5 章，中國中醫學會，2012 年。

--------------------------------------------------



INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Question: Among the four stages of Wei Qi Ying Xue, which stage is described as the deepest and most severe, and is associated with Liver and Kidney pathology? 

System Response:
根據中醫理論，Wei Qi Ying Xue 的四個階段是：

1. Shao Yang (少陽)：表現為表面症狀，如皮膚病變、頭痛等。
2. Tai Yang (太陽)：表現為全身症狀，如發燒、頭痛、肌肉疼痛等。
3. Shao Yin (少陰)：表現為內在症狀，如心悸、胸痛、脫屑等。
4. Tai Yin (太陰)：表現為最深層的症狀，如肝腎功能衰竭、身體無力、智力下降等。

根據這些描述，Tai Yin (太陰)階段是最深層的階段，與肝腎功能衰竭有關。因此，答案是 Tai Yin (太陰)階段。


［常識］中醫理論中，Wei Qi Ying Xue 的四個階段是描述病理變化的階段，各階段的特徵和表現方式不同。


參考資料：
Wei Qi Ying Xue 的四個階段描述見於《金匱要略》等中醫經典著作。

--------------------------------------------------



In [9]:
# 5. 計算 Accuracy
overall_acc = test_df["is_correct"].mean()

print("\n=== 每個 domain 的 Accuracy ===")
domain_stats = (
    test_df.groupby("domain")["is_correct"]
           .agg(["count", "sum"])
           .reset_index()
           .rename(columns={"sum": "correct"})
)
domain_stats["accuracy"] = domain_stats["correct"] / domain_stats["count"]
print(domain_stats.to_string(index=False, 
      formatters={"accuracy": "{:.2%}".format}))

print(f"\nOVERALL Accuracy = {overall_acc:.2%}")


=== 每個 domain 的 Accuracy ===
domain  count  correct accuracy
   CCD     39       23   58.97%
    中醫     60       25   41.67%

OVERALL Accuracy = 48.48%


In [53]:
# excel轉成dataset
import pandas as pd
import json

# 讀取CSV檔案 (假設檔案已上傳或路徑已設定)
df = pd.read_csv('results/baseline_48.csv')  # 請替換成你的CSV檔案路徑

# 初始化dataset列表
dataset = []

# 遍歷DataFrame的每一行
for index, row in df.iterrows():
    # 取得需要的欄位，如果不存在則設為空字串
    q = row.get('question', '') if not pd.isna(row.get('question', '')) else ''
    resp = row.get('llm_response', '') if not pd.isna(row.get('llm_response', '')) else ''
    ref_ctx = row.get('RAG', '') if not pd.isna(row.get('RAG', '')) else ''
    gt = row.get('answers', '') if not pd.isna(row.get('answers', '')) else ''

    
    # 將資料加入dataset
    entry = {
        "user_input": str(q),
        "response": str(resp),
        "reference_contexts": [ref_ctx] if isinstance(ref_ctx, str) and ref_ctx else [],
        "reference": gt
    }
    
    dataset.append(entry)

# 顯示結果的第一筆資料作為範例
if dataset:
    print("已轉換的第一筆資料範例:")
    print(json.dumps(dataset[0], ensure_ascii=False, indent=2))

# 儲存為JSON檔案 (選用)
# with open('output_dataset.json', 'w', encoding='utf-8') as f:
#     json.dump(dataset, f, ensure_ascii=False, indent=2)
# print("已儲存資料至 output_dataset.json")

# 回傳dataset變數供後續使用
# dataset

已轉換的第一筆資料範例:
{
  "user_input": "根據中獸醫的概念，「虛證 (Deficiency Pattern)」通常出現在年輕且健康的動物身上",
  "response": "根據中醫理論，「虛證型」（Deficiency Pattern）通常見於老年或健康的動物。這與TCVM（Traditional Chinese Veterinary Medicine）的觀點相符，它強調了個體的內在平衡和氣血狀態。\n\n然而，這個敘述並不完全準確。「虛證型」其實更常見於中老年或身體衰弱的動物，而不是所有健康的年輕動物。這些動物可能因為多種原因導致內在氣血不足，從而出現虛證型的症狀。\n\n因此，答案是：\n\nFalse\n\n［常識］：TCVM認為「虛證型」是由於個體的氣血狀態不佳所導致，通常見於老年或身體衰弱的動物，而不是所有健康的年輕動物。 [1]\n\n參考資源：\n\n[1] Chinese Veterinary Materia Medica.pdf",
  "reference_contexts": [
    "Deficiency Patterns are often seen in geriatric patients with chronic illnesses.The Deficiency Pattern is often seen in older patients with chronic illnesses."
  ],
  "reference": "FALSE"
}


RAGAs

In [54]:
from dotenv import load_dotenv
import os
from pathlib import Path

env_path = Path("key") / ".env"
load_dotenv(dotenv_path=env_path, override=False)

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("找不到 OPENAI_API_KEY")

In [55]:
from ragas.llms import LangchainLLMWrapper
from langchain_openai import ChatOpenAI  # langchain>=0.1
llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0)
evaluator_llm = LangchainLLMWrapper(llm)

In [56]:
from ragas import EvaluationDataset
evaluation_dataset = EvaluationDataset.from_list(dataset)

In [57]:
from ragas import evaluate
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness,ContextPrecision

result = evaluate(
    dataset=evaluation_dataset,
    metrics=[
        # LLMContextRecall(),
        # ContextPrecision(),
        # Faithfulness(),      #only QA 忠實度
        FactualCorrectness(), #only QA 正確性
    ],
    llm=evaluator_llm
)


Evaluating:   0%|          | 0/99 [00:00<?, ?it/s]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ap

In [58]:
result

{'factual_correctness(mode=f1)': 0.6294}

In [59]:
result.scores

[{'factual_correctness(mode=f1)': 0.2},
 {'factual_correctness(mode=f1)': 0.91},
 {'factual_correctness(mode=f1)': 1.0},
 {'factual_correctness(mode=f1)': 0.95},
 {'factual_correctness(mode=f1)': 0.89},
 {'factual_correctness(mode=f1)': 0.0},
 {'factual_correctness(mode=f1)': 0.88},
 {'factual_correctness(mode=f1)': 0.77},
 {'factual_correctness(mode=f1)': 0.0},
 {'factual_correctness(mode=f1)': 0.0},
 {'factual_correctness(mode=f1)': 1.0},
 {'factual_correctness(mode=f1)': 0.86},
 {'factual_correctness(mode=f1)': 0.0},
 {'factual_correctness(mode=f1)': 0.92},
 {'factual_correctness(mode=f1)': 0.0},
 {'factual_correctness(mode=f1)': 0.96},
 {'factual_correctness(mode=f1)': 0.0},
 {'factual_correctness(mode=f1)': 0.62},
 {'factual_correctness(mode=f1)': 1.0},
 {'factual_correctness(mode=f1)': 0.6},
 {'factual_correctness(mode=f1)': 1.0},
 {'factual_correctness(mode=f1)': 0.82},
 {'factual_correctness(mode=f1)': 0.92},
 {'factual_correctness(mode=f1)': 0.5},
 {'factual_correctness(mode=f

In [60]:
df['factual_correctness'] = [item['factual_correctness(mode=f1)'] for item in result.scores]
df

,id,actor,type,domain,question,query_for_embed,answers,RAG,pdf_file,page,llm_response,predicted,is_correct,factual_correctness
0,1,飼主,true_false,中醫,根據中獸醫的概念，「虛證 (Deficiency Pattern)」通常出現在年輕且健康的動物身上,"According to TCVM concepts, ""Deficiency Patter...",FALSE,Deficiency Patterns are often seen in geriatri...,Herbology_ocr.pdf,563,根據中醫理論，「虛證型」（Deficiency Pattern）通常見於老年或健康的動物。這...,FALSE,1,0.20
1,2,飼主,multiple_choice,中醫,在中獸醫的「八綱辨證 (Eight Principles)」中，哪一對概念描述了疾病的深淺層...,"In the TCVM ""Eight Principles"" method, which p...",C,The Eight Principles method further characteri...,Herbology_ocr.pdf,15,根據TCVM的八則原理，我們可以知道，描述疾病深度的概念是「外在」和「內在」。這兩個概念描述...,C,1,0.91
2,3,飼主,true_false,中醫,如果動物表現出全身無力、疲勞和脈搏虛弱，這可能與「虛證 (Deficiency Patter...,"If an animal shows general weakness, fatigue,...",TRUE,Patients with Defi ciency Patterns usually exh...,Herbology_ocr.pdf,15,根據我的知識和常識，我可以回答這個問題。\n\n答案：True［2］\n\n理由：在中醫理論...,TRUE,1,1.00
3,4,飼主,true_false,中醫,根據資料，草藥「百合 (Bai He)」，又稱「Lily bulb」，在古代法國曾代表王室的威嚴,"The herb ""百合Bai He"", also known as ""Lily bulb""...",TRUE,"In ancient France, Bai He repre-sented the maj...",Herbology_ocr.pdf,563,這個問題中提到的藥物是來自中國的傳統中醫藥，與古法國皇室無關。\n\n根據我的知識，百合（B...,FALSE,0,0.95
4,5,飼主,multiple_choice,中醫,根據歷史故事，哪種草藥據說幫助諸葛亮的軍隊在飢餓寒冷時感到溫暖並嚇跑敵人（儘管帶來了尷尬的副...,"According to a historical story , which herb w...",B,Chapter 1: Herbs to Tonify Defi ciency 61\n4. ...,Herbology_ocr.pdf,15,根據題目中描述的情景，似乎是在提到一種古代中藥物的故事。 Zhuge Liang是一位中國歷...,UNK,0,0.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,105,飼主,true_false,CCD,環境刺激增加可延緩 CCD 退化。,Increased environmental stimulation can delay ...,TRUE,Cognitive enrichment has been shown to delay c...,‎Canine Cognitive Dysfunction Pathophysiology ...,13,這個問題很好！根據我的知識和經驗，我可以說：\n\n**False**\n\n犬認知功能障礙...,FALSE,0,0.92
95,106,飼主,true_false,CCD,若犬夜吠屬 CCD，一定無法改善。,If the genus CCD of the genus genus is definit...,FALSE,Sleep-wake disturbances可透過多種治療策略改善.,‎Canine Cognitive Dysfunction Pathophysiology ...,12,對於犬認知功能障礙綜合症（CCD），我需要更多資訊來回答這個題目。但根據我的知識，我可以確認...,FALSE,1,0.22
96,107,飼主,true_false,CCD,狗在水碗前發呆可能是 CCD 警訊之一。,The dog is dazed in front of the water bowl ma...,TRUE,Standing over the water bowl but not drinking ...,‎Under diagnosis of canine cognitive dysfuncti...,4,這個描述聽起來很熟悉。犬認知功能障礙綜合症（CCD）是一種常見的中老年犬疾病，可能導致各種類...,TRUE,1,1.00
97,108,飼主,true_false,CCD,CCD 只會在 14 歲以上犬隻才出現。,CCD only appears in dogs over the age of 14.,FALSE,Prevalence 3.4 % already seen in 8–10-year-old...,‎Under diagnosis of canine cognitive dysfuncti...,3,CCD（犬認知功能障礙綜合症）的典型年齡範圍為5-13歲之間，尤其是老年犬。這種疾病不僅限於...,FALSE,1,0.00


In [62]:
# ==== 建立輸出資料夾 ====
from pathlib import Path
OUT_DIR = Path("results")
OUT_DIR.mkdir(exist_ok=True)

# ==== 存檔 ====
csv_path = OUT_DIR / "baseline_48_ragas.csv"
df.to_csv(csv_path, index=False, encoding="utf-8-sig")
print(f"✅ Baseline 結果已存到 {csv_path}")


✅ Baseline 結果已存到 results/baseline_48_ragas.csv
